### 事前準備

**TensorFlow1.xのセットアップ**
ColabのDefault Versionが2.0になった時の対応用。<font color=red>**ランタイムの再起動が必要!!**</font>

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

GoogleDriveのマウント

In [0]:
from google.colab import drive
drive.mount('/content/drive')

PC-Cameraの準備

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# **Open Poseを使った骨格推定（既成のモデルを使った手軽なAI）**


<table><tr>
<td><img src="https://www.fujitsu.com/jp/imagesgig5/pict_38_tcm102-4508765_tcm102-2750236-32.png" width="100"></td>
<td><h1><b>【ご参考】Zinraiによる行動分析ソリューション</b></h1></td>
</tr></table> 

**<font color=red>※OpenPoseでは無いが、Zinraiでも骨格推定を用いた行動分析ソリューションを提供しています。</font>**

> [人の映像から、動作や行動をディープラーニングで認識します。
「歩く」「首を振る」「手を伸ばす」など約100種類の基本動作を90%以上の精度で認識する学習済みモデルを用い、これら基本動作を組み合わせることで、不審行動や購買行動といった人の複雑な行動を認識させることが可能です。](https://www.fujitsu.com/jp/solutions/business-technology/ai/ai-zinrai/technology/index.html#img-03)   \\


### OpenPoseとは
> OpenPoseは、コンピュータビジョンに関する国際学会CVPR2017でCMU（カーネギーメロン大学）が発表した、keypoint（特徴点）の検出とkeypoint同士の関係の推定を行う技術です。OpenPoseを使うと、関節の位置など、人の体における特徴点がどの座標にあるかが分かります。

### **セットアップ(OpenPose)**

OpenPoseをセットアップする。（だいたい 3～4分くらいかかる）

In [0]:
# SWIG を準備
!apt-get -q -y install swig
# TF-openposeをクローン
!git clone https://www.github.com/ildoonet/tf-openpose
# クローンしたディレクトリに入る
%cd tf-openpose
# openpose動作のための、ライブラリをインストール
!pip3 install -r requirements.txt
# Openposeのモデルをダウンロード
%cd models/graph/cmu
!bash download.sh
%cd ../../../
# pafprocessをインストール
%cd tf_pose/pafprocess
!ls
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
%cd ../../

run.pyの若干カスタマイズ版run_new.py(FileSaveを追加)をGoogleDriveからコピー

In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/run_new.py" "/content/tf-openpose/run_new.py"

## **写真画像から骨格を推定**

**PC-Cameraを起動**

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  display(Image(filename))
except Exception as err:
  print(str(err))

**撮影画像から骨格を推定(OpenPose)**

In [0]:
#%run -i run.py --model=mobilenet_thin --image=/content/tf-openpose/photo.jpg
#!cd /content/tf-openpose
%run -i run_new.py --model=mobilenet_thin --image=/content/tf-openpose/photo.jpg

In [0]:
  # 骨格推定結果の表示（参考）
  display(Image("/content/tf-openpose/result.jpg"))

In [0]:
!cd /content/tf-openpose/

# 番外：動画からOpenPoseで骨格推定

**ちょーおおまかな概要** \\
> 動画データから1枚ずつフレームを切り出し、切り出した画像を一枚ずつ骨格推定して、最後にそれをつなぎ合わせて動画にする。。。 \\
サンプルは、YouTubeから拝借。。（Zinrai関連＝ノクリアCM、富士通チア、エフサスTimeCreatorのCM）


**サンプル動画の準備**：GoogleDriveに予め用意した動画を、Colabの/tf-openpose配下にコピー

In [0]:
!cp '/content/drive/My Drive/dance.mp4' "/content/tf-openpose/dance.mp4"
!cp '/content/drive/My Drive/FSAS_TC_30S.mp4' "/content/tf-openpose/FSAS_TC_30S.mp4"
!cp '/content/drive/My Drive/FJ_Cheer.mp4' "/content/tf-openpose/FJ_Cheer.mp4"
!cp '/content/drive/My Drive/FJ_ZinraiCM.mp4' "/content/tf-openpose/FJ_ZinraiCM.mp4"
!pwd
!ls

**動画の確認（再生）**：うまく再生できない時がある（その時はGoogleDriveの元動画をプレビュー）

In [0]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/tf-openpose/Out_dance.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

**動画で骨格推定し動画で保存**：動画で骨格推定し、/tf-openpose配下にOut_dance.mp4で保存

##### **Sample1：富士通エフサス TimeCreatorのCM動画**

In [0]:
import argparse
import logging
import time
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
movie_name = 'FSAS_TC_30S'
img_outdir = './img'
os.makedirs(img_outdir, exist_ok=True)
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')  # 動画作成
video  = cv2.VideoWriter('Out_dance.mp4', fourcc, 10.0, (1280, 720))
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
    outimg_files = []
    count = 0
    w = 640 
    h = 360
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))
    cap = cv2.VideoCapture('FSAS_TC_30S.mp4')       # 動画読み込み
    while True:         # 動画用の画像作成
        ret, image = cap.read()
        if ret == True:
            count += 1                # １フレームずつ処理
            if count % 100 == 0:
                print('Image No.：{0}'.format(count))
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)               # 画像出力
            cv2.imwrite(outimg_file, image)
            video.write(image)       
        else:
            break
    video.release()

##### **Sample2：富士通ゼネラルのノクリアCM（ノクリア Powered by Zinrai）の動画**

In [0]:
import argparse
import logging
import time
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
movie_name = 'FJ_ZinraiCM'
img_outdir = './img'
os.makedirs(img_outdir, exist_ok=True)
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')  # 動画作成
video  = cv2.VideoWriter('Out_dance.mp4', fourcc, 10.0, (640, 360))
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
    outimg_files = []
    count = 0
    w = 640 
    h = 360
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))
    cap = cv2.VideoCapture('FJ_ZinraiCM.mp4')       # 動画読み込み
    while True:         # 動画用の画像作成
        ret, image = cap.read()
        if ret == True:
            count += 1                # １フレームずつ処理
            if count % 100 == 0:
                print('Image No.：{0}'.format(count))
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)               # 画像出力
            cv2.imwrite(outimg_file, image)
            video.write(image)       
        else:
            break
    video.release()

##### **Sample3：富士通フロンティアーズのチア動画**

In [0]:
import argparse
import logging
import time
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
movie_name = 'FJ_Cheer'
img_outdir = './img'
os.makedirs(img_outdir, exist_ok=True)
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')  # 動画作成
video  = cv2.VideoWriter('Out_dance.mp4', fourcc, 10.0, (426, 240))
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
    outimg_files = []
    count = 0
    w = 640 
    h = 360
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))
    cap = cv2.VideoCapture('FJ_Cheer.mp4')       # 動画読み込み
    while True:         # 動画用の画像作成
        ret, image = cap.read()
        if ret == True:
            count += 1                # １フレームずつ処理
            if count % 100 == 0:
                print('Image No.：{0}'.format(count))
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)               # 画像出力
            cv2.imwrite(outimg_file, image)
            video.write(image)       
        else:
            break
    video.release()

**骨格推定した動画をGoogleDriveにコピー**：結果を確認するために、GoogleDriveにコピーする \\
※ファイル反映にラグあり。何度かコピーしてcolabのエクスプローラで反映されているか確認する※

##### **骨格推定した動画の確認（再生）**：GoogleDriveにコピーして、動画を再生。<font color=red><b>（GoogleDriveの方からプレビューする）</b></font>

In [0]:
!cp "/content/tf-openpose/Out_dance.mp4" "/content/drive/My Drive/Out_dance.mp4"

**<後処理>ワークファイル(/tf-openpose/img配下の骨格推定後フレームjpeg)の削除**

In [0]:
!rm -r /content/tf-openpose/img/*
!ls -x /content/tf-openpose/img

# 番外：PoseNetというのもあります（ご参考）

**PoseNet** \\
> Googleが開発したWebブラウザでリアルタイムに人間の姿勢推定を可能にする機械学習モデル。　 \\
映像中の人物から1つのポーズまたは複数のポーズを検出できる。   \\
詳細は、https://github.com/tensorflow/tfjs-models/tree/master/posenet

JavaScriptで動作し、WebCamも使える。公式サイトでデモ公開しているので、ちょっとみましょう！ \\
**[デモサイト](https://storage.googleapis.com/tfjs-models/demos/posenet/camera.html)**
